In [1]:
import pandas as pd
import numpy as np

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

import joblib

from data_preprocessing import pipeline
from feature_preprocessing import pipeline2

In [2]:
import matplotlib.pyplot as plt
plt.rcParams['font.family'] ='Malgun Gothic'
plt.rcParams['axes.unicode_minus'] =False

### 데이터 전처리

In [3]:
# 로우 데이터 불러오기
file_path = "./storage/raw_data/병합_청약매물_목록_정보_픽스2.csv"
# df = pd.read_csv(file_path, encoding ='cp949')
df = pd.read_csv(file_path)

# 데이터 전처리````
preprocessing_pipeline = pipeline(type='train')
df = preprocessing_pipeline.transform(df)

# 파일 저장
file_version = "3트_시세차익X"
output_file = f"./storage/train_data/train-{file_version}.csv"
df.to_csv(output_file, index=False, encoding='cp949')

C:\Users\user\AppData\Local\Temp\ipykernel_13472\3828398876.py:4: DtypeWarning: Columns (48) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
c:\Users\user\mid_project\src\data_preprocessing.py:146: DtypeWarning: Columns (36,48) have mixed types. Specify dtype option on import or set low_memory=False.
  df_estate_price = pd.read_csv(csv_path, encoding="cp949")
c:\Users\user\mid_project\src\data_preprocessing.py:329: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['최저당첨가점'

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2981 entries, 44 to 17246
Data columns (total 27 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   공급지역코드        2981 non-null   int64  
 1   공급규모          2981 non-null   int64  
 2   투기과열지구        2981 non-null   object 
 3   조정대상지역        2981 non-null   object 
 4   분양가상한제        2981 non-null   object 
 5   정비사업          2981 non-null   object 
 6   공공주택지구        2981 non-null   object 
 7   대규모택지개발지구     2981 non-null   object 
 8   수도권내민영공공주택지구  2981 non-null   object 
 9   공급세대수         2981 non-null   int64  
 10  순위            2981 non-null   object 
 11  거주지역          2981 non-null   object 
 12  접수건수          2981 non-null   int64  
 13  경쟁률           2981 non-null   float64
 14  최저당첨가점        2981 non-null   float64
 15  토픽 1          2981 non-null   float64
 16  토픽 2          2981 non-null   float64
 17  토픽 3          2981 non-null   float64
 18  토픽 4          2981 non-null   f

### 모델 학습 및 평가

In [ ]:
# 전처리된 데이터 가져오기

file_version = "250317-01"
file_path = f"./storage/train_data/t``ain-{file_version}.csv"
df = pd.read_csv(file_path, encoding='cp949')

# df = pd.read_csv("./storage/train_data/train-2트_시세차익o.csv", encoding = 'cp949')

In [6]:
df

,공급지역코드,공급규모,투기과열지구,조정대상지역,분양가상한제,정비사업,공공주택지구,대규모택지개발지구,수도권내민영공공주택지구,공급세대수,...,토픽 3,토픽 4,토픽 5,토픽 6,토픽 7,전용면적,공급금액(최고가 기준),법정동코드,거래금액(만원),전용면적당 시세차익
0,400,29,N,N,N,Y,N,N,N,5,...,0.001200,0.001167,0.050333,0.001200,0.924133,59.9393,4.497000e+08,2.817710e+09,16300.000000,7.502304e+06
1,400,29,N,N,N,Y,N,N,N,4,...,0.001200,0.001167,0.050333,0.001200,0.924133,74.9227,5.675000e+08,2.817710e+09,16300.000000,7.574187e+06
2,100,482,Y,Y,Y,Y,N,N,N,35,...,0.001267,0.719333,0.002967,0.001267,0.037367,59.9800,1.724700e+09,1.165010e+09,203726.315789,2.875227e+07
3,100,482,Y,Y,Y,Y,N,N,N,30,...,0.001267,0.719333,0.002967,0.001267,0.037367,59.9500,1.760600e+09,1.165010e+09,203726.315789,2.936549e+07
4,100,482,Y,Y,Y,Y,N,N,N,14,...,0.001267,0.719333,0.002967,0.001267,0.037367,59.7800,1.796500e+09,1.165010e+09,203726.315789,3.004954e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2976,410,747,N,N,Y,N,Y,Y,Y,161,...,0.000900,0.517567,0.168600,0.007433,0.085333,84.1316,4.252000e+08,4.139013e+09,23772.471910,5.053664e+06
2977,410,747,N,N,Y,N,Y,Y,Y,57,...,0.000900,0.517567,0.168600,0.007433,0.085333,84.3852,4.231000e+08,4.139013e+09,23772.471910,5.013590e+06
2978,410,747,N,N,Y,N,Y,Y,Y,57,...,0.000900,0.517567,0.168600,0.007433,0.085333,84.3852,4.231000e+08,4.139013e+09,23772.471910,5.013590e+06
2979,410,747,N,N,Y,N,Y,Y,Y,216,...,0.000900,0.517567,0.168600,0.007433,0.085333,84.3832,4.272000e+08,4.139013e+09,23772.471910,5.062296e+06


In [ ]:
X = df.drop(columns=["최저당첨가점"])
Y = df["최저당첨가점"]

X_train, X_test, y_train, y_test = train_test_split(
    X,  # 타겟(최저당첨가점) 제외
    Y,  # 타겟
    test_size=0.2,
    random_state=42
)

In [4]:
feature_pipeline = pipeline2()

# 학습 데이터(X_train)에 fit
feature_pipeline.fit(X_train)

# 학습 데이터(X_train)를 변환
X_train_transformed = feature_pipeline.transform(X_train)

# 테스트 데이터(X_test)를 변환
X_test_transformed = feature_pipeline.transform(X_test)

Index(['공급지역코드', '공급규모', '투기과열지구', '조정대상지역', '분양가상한제', '정비사업', '공공주택지구',
       '대규모택지개발지구', '수도권내민영공공주택지구', '공급세대수', '순위', '거주지역', '접수건수', '경쟁률',
       '토픽 1', '토픽 2', '토픽 3', '토픽 4', '토픽 5', '토픽 6', '토픽 7', '법정동코드',
       '시세차익'],
      dtype='object')
Index(['공급규모', '공급세대수', '접수건수', '경쟁률', '토픽 1', '토픽 2', '토픽 3', '토픽 4', '토픽 5',
       '토픽 6', '토픽 7', '법정동코드', '시세차익', '투기과열지구_N', '투기과열지구_Y', '조정대상지역_N',
       '조정대상지역_Y', '분양가상한제_N', '분양가상한제_Y', '정비사업_N', '정비사업_Y', '공공주택지구_N',
       '공공주택지구_Y', '대규모택지개발지구_N', '대규모택지개발지구_Y', '수도권내민영공공주택지구_N',
       '수도권내민영공공주택지구_Y', '순위_1순위', '거주지역_기타지역', '거주지역_해당지역', '공급지역코드_100',
       '공급지역코드_400', '공급지역코드_410'],
      dtype='object')
⚠️ Warning: 새로운 법정동코드 발견 {'4115010900.0', '2817710200.0', '4183033022.0', '1121510400.0', '4183031023.0'}. 'unknown'으로 대체합니다.
Index(['공급지역코드', '공급규모', '투기과열지구', '조정대상지역', '분양가상한제', '정비사업', '공공주택지구',
       '대규모택지개발지구', '수도권내민영공공주택지구', '공급세대수', '순위', '거주지역', '접수건수', '경쟁률',
       '토픽 1', '토픽 2', '토픽 3', '토픽 4', '

c:\Users\user\anaconda3\envs\project-env\Lib\site-packages\sklearn\pipeline.py:62: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(
c:\Users\user\anaconda3\envs\project-env\Lib\site-packages\sklearn\pipeline.py:62: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(


### 학습

In [1]:
# 최고당첨금액 , 최저당첨금액
'''
Randomforest, XGB, LGB 비교결과
XGB가 가장 높았지만, 성능평가에서 0.1정도의 아주 작은 차이라 더 가볍고 빠르게 돌아가는 모델인 LightGBM 이용 제안
추후 결정되면 코드 적어놓겠음
'''

'\nRandomforest, XGB, LGB 비교결과\nXGB가 가장 높았지만, 성능평가에서 0.1정도의 아주 작은 차이라 더 가볍고 빠르게 돌아가는 모델인 LightGBM 이용 제안\n추후 결정되면 코드 적어놓겠음\n'

### 모델 저장 

In [10]:
# 모델 저장 및 로드

version = 'rf_grid_0.0.1'

# 모델 저장
joblib.dump(rf_model, f"./storage/trained_model/model_{version}.pkl")

['./storage/trained_model/model_rf_grid_0.0.1.pkl']

In [8]:
# 모델 로드
loaded_model = joblib.load("./storage/trained_model/model_0.0.1.pkl")

# 예측
# X_test = np.array([[1, 2]])
# prediction = loaded_model.predict(X_test)
# print("Prediction:", prediction)